## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import acquire
import prepare
import wrangle

import warnings
warnings.simplefilter('ignore')

## Acquire

In [245]:
# Acquire cohort data from Codeups MySQL database and cache csv
df = acquire.get_cohort_data()

In [246]:
# Acquire cohort data from local cache, if not present must download from github `cohorts_information - cohorts.csv`
info_df = acquire.get_cohort_information_data()

In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        847330 non-null  object 
 1   time        847330 non-null  object 
 2   path        847329 non-null  object 
 3   user_id     847330 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          847330 non-null  object 
 6   id          72405 non-null   float64
 7   name        72405 non-null   object 
 8   slack       72405 non-null   object 
 9   start_date  72405 non-null   object 
 10  end_date    72405 non-null   object 
 11  created_at  72405 non-null   object 
 12  updated_at  72405 non-null   object 
 13  deleted_at  0 non-null       float64
 14  program_id  72405 non-null   float64
dtypes: float64(4), int64(1), object(10)
memory usage: 103.4+ MB


In [248]:
df.describe()

,user_id,cohort_id,id,deleted_at,program_id
count,847330.000000,847330.000000,72405.000000,0.0,72405.000000
mean,456.707344,48.501049,30.015248,NaN,1.552876
std,250.734201,32.795482,35.619910,NaN,0.662231
min,1.000000,1.000000,1.000000,NaN,1.000000
25%,263.000000,28.000000,11.000000,NaN,1.000000
50%,476.000000,33.000000,12.000000,NaN,1.000000
75%,648.000000,57.000000,53.000000,NaN,2.000000
max,981.000000,139.000000,139.000000,NaN,4.000000


##### Summary Takeaways:
- date, end_date, and start_date will become datetime as datetime type
- cohort_id will be cast as int
- id will be cast as int
- deleted_at will be dropped
- Will clean for null values

In [6]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8,97.105.19.61,1.0,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22,97.105.19.61,2.0,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0


In [7]:
info_df

,name,start_date,end_date,program_id
cohort_id,,,,
1,Arches,2014-02-04,2014-04-22,1
2,Badlands,2014-06-04,2014-08-22,1
3,Carlsbad,2014-09-04,2014-11-05,1
4,Denali,2014-10-20,2015-01-18,1
5,Everglades,2014-11-18,2015-02-24,1
6,Franklin,2015-02-03,2015-05-26,1
7,Glacier,2015-06-05,2015-10-06,1
8,Hampton,2015-09-22,2016-02-06,1
9,Apollo,2015-03-30,2015-07-29,4


In [249]:
info_df.shape()

(52, 4)

In [13]:
df.shape

(847330, 15)

## Prepare/Wrangle

##### Wrangle/Prepare Summary
- Acquires the cohort data and cohort information data CSVs and joins the dataframes on cohort_id.
- Change dtypes where necessary
- Combined the date and time column into a datetime, and converted the datatype to datetime.
- Placed my new datetime column to the index for later use.
- Replaced the program_id with its corresponding program name and subdomain.
- Dropped unnecessary columns 'date','time','deleted_at','program_id','id'.
- Renamed 'name' to 'cohort_name' for my personal readability.
- Created a dictionary to map cohort name to cohort id.
- Filled nulls in cohort_id.

In [256]:
# Acquire and Prepare data
df = wrangle.wrangle_cohort_data()

In [257]:
# Null Check
df.isnull().sum()

path                 0
user_id              0
cohort_id            0
ip                   0
cohort_name          0
start_date           0
end_date             0
program_id           0
datetime             0
count_helper         0
program_subdomain    0
slack                0
subpath              0
dtype: int64

In [258]:
df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,datetime,count_helper,program_subdomain,slack,subpath
0,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,[java-ii]
1,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[java-ii, object-oriented-programming]"
2,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[slides, object_oriented_programming]"
3,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[javascript-i, functions]"
4,java-i,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,[java-i]


In [259]:
# Defining shape pre-preparation
pre_shape = df.shape
pre_shape

(801475, 13)

#### Further Cleaning/Prep:

In [260]:
# Cleaning df further to remove anything that is not a lesson

# If end is jpg or jpeg, remove from df
df = df[~df.path.str.endswith('jpg')] 
df = df[~df.path.str.endswith('jpeg')]
df = df[~df.path.str.endswith('svg')]

# If end is an index or stem is a / remove from df
df = df[(df.path != '/') & (~df.path.str.endswith('search_index.json'))\
        & (~df.path.str.endswith('index.html'))]

In [261]:
df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,datetime,count_helper,program_subdomain,slack,subpath
0,java-ii,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,[java-ii]
1,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[java-ii, object-oriented-programming]"
2,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[slides, object_oriented_programming]"
3,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[javascript-i, functions]"
4,java-i,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,[java-i]


#### Creating Subpath/Lesson:
> - Will parse path, separating by / in the path.
> - Will remove uneccessary data
> - Filtered df down to only entries considered to be lessons
>     - Lesson = path.len() >= 2

In [262]:
# Creating subpath column
df['subpath'] = df.path.apply(lambda x: x.split('/'))

In [263]:
# Getting stems and destination from paths
sub_lens = [] # lengths of each path
large_paths = [] # paths larger than 1 /'s deep
starter = [] # path stem
final = [] # path tail
lesson = [] # tails where the stem != 'content'
for path in df.subpath.values:
    sub_lens.append(len(path))
    starter.append(path[0])
    final.append(path[-1])
    if len(path) >= 2:
        large_paths.append(path)
    if path[0] == 'content': # checking for content stem
        lesson.append(path[-1])
    else:
        lesson.append('None') # if stem != content cast as None

In [264]:
# Creating stem and finisher column from subpaths
df['path_stem'] = starter
df['path_tail'] = final
df['content'] = lesson

In [265]:
# Recasting df without any paths len < 2
df = df[df.subpath.isin(large_paths)]

In [267]:
df.head()

,path,user_id,cohort_id,ip,cohort_name,start_date,end_date,program_id,datetime,count_helper,program_subdomain,slack,subpath,path_stem,path_tail,content
1,java-ii/object-oriented-programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[java-ii, object-oriented-programming]",java-ii,object-oriented-programming,None
2,slides/object_oriented_programming,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[slides, object_oriented_programming]",slides,object_oriented_programming,None
3,javascript-i/functions,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-26,1,php,#hampton,"[javascript-i, functions]",javascript-i,functions,None
6,teams/13,1,8,72.177.226.58,Hampton,2015-09-22,2016-02-06,1,2018-01-27,1,php,#hampton,"[teams, 13]",teams,13,None
7,javascript-i/conditionals,1,8,97.105.19.61,Hampton,2015-09-22,2016-02-06,1,2018-01-29,1,php,#hampton,"[javascript-i, conditionals]",javascript-i,conditionals,None


Most common stem for lessons is content

#### Checking for indescrepencies in path

In [28]:
# Noticed a few entries have strange github paths
for path in df.subpath.values:
    for i in path:
        if (i == '%20https:') | (i == ',%20https:'):
            print(path)

['%20https:', '', 'github.com', 'RaulCPena']
[',%20https:', '', 'github.com', 'RaulCPena']


Two instances of a path leading to someones personal github. Will remove these entries as they dont lead to any lessons.

In [29]:
# Masking for where the path is equal to the github link
mask = (df.path != '%20https://github.com/RaulCPena') & (df.path != ',%20https://github.com/RaulCPena')

In [30]:
# Using mask to remove rows where path is like %20https://github.com/RaulCPena
df = df[mask]

In [31]:
# Final check for the anomalous entries
for path in df.subpath.values:
    for i in path:
        if (i == '%20https:') | (i == ',%20https:'):
            print(path)

In [268]:
# How many rows dropped?
print(f'Number of rows dropped: {pre_shape[0] - df.shape[0]}')

Number of rows dropped: 182722


In [273]:
df.shape

(618753, 16)

#### Separating the dataframes by subdomain:

In [269]:
# PHP Full Stack path
php_df = df[df.program_subdomain == 'php']

In [270]:
# Java path
java_df = df[df.program_subdomain == 'java']

In [271]:
# Data science
ds_df = df[df.program_subdomain == 'ds']

##### Post-prep Summary:
- Removed nulls
- Dropped all entries not leading to lesson
- Removed some anomalous path entries
- Created subsets for each subdomain
- Final shape: Dataframe has 16 columns and 618753 rows after cleaning.

# Questions

### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [36]:
# Creating grouped df by program_subdomain and path value_counts
lesson_df = pd.DataFrame(df.groupby('program_subdomain').path.value_counts())

In [37]:
# Renaming to be accurate to info and reseting multi-index
lesson_df = lesson_df.rename(columns = {'path': 'visits'}, level = 0).reset_index()

In [38]:
# Getting max values for each subdomain for each max value
ds_max = lesson_df[(lesson_df.program_subdomain == 'ds')].visits.max()
php_max = lesson_df[(lesson_df.program_subdomain == 'php')].visits.max()
fe_max = lesson_df[(lesson_df.program_subdomain == 'fe')].visits.max()
java_max = lesson_df[(lesson_df.program_subdomain == 'java')].visits.max()

In [39]:
# Creating subpath column
lesson_df['subpath'] = lesson_df.path.apply(lambda x: x.split('/'))

In [40]:
lesson_df = lesson_df[['program_subdomain', 'path', 'subpath', 'visits']]
lesson_df.head()

,program_subdomain,path,subpath,visits
0,ds,classification/overview,"[classification, overview]",1785
1,ds,1-fundamentals/1.1-intro-to-data-science,"[1-fundamentals, 1.1-intro-to-data-science]",1633
2,ds,sql/mysql-overview,"[sql, mysql-overview]",1424
3,ds,fundamentals/intro-to-data-science,"[fundamentals, intro-to-data-science]",1413
4,ds,6-regression/1-overview,"[6-regression, 1-overview]",1124


In [41]:
# Getting stems and destination from paths
sub_lens = [] # lengths of each path
large_paths = [] # paths larger than 2 /'s deep
starter = [] # path stem
final = [] # path tail
lesson = [] # tails where the stem != 'content'
for path in lesson_df.subpath.values:
    sub_lens.append(len(path))
    starter.append(path[0])
    final.append(path[-1])
    if len(path) > 2:
        large_paths.append(path)
    if path[0] == 'content': # checking for content stem
        lesson.append(path[-1])
    else:
        lesson.append('None') # if stem != content cast as None

In [42]:
# Creating columns for stems, tails and content containing entries
lesson_df['stem'] = starter
lesson_df['tail'] = final
lesson_df['content'] = lesson
lesson_df = lesson_df[['program_subdomain', 'path', 'stem', 'tail', 'content', 'visits']]

In [43]:
lesson_df.head()

,program_subdomain,path,stem,tail,content,visits
0,ds,classification/overview,classification,overview,None,1785
1,ds,1-fundamentals/1.1-intro-to-data-science,1-fundamentals,1.1-intro-to-data-science,None,1633
2,ds,sql/mysql-overview,sql,mysql-overview,None,1424
3,ds,fundamentals/intro-to-data-science,fundamentals,intro-to-data-science,None,1413
4,ds,6-regression/1-overview,6-regression,1-overview,None,1124


In [44]:
# Masking to find each subdmoains most visited lesson
mask = (lesson_df.visits == php_max)  | (lesson_df.visits == java_max) | ((lesson_df.visits == fe_max) & (lesson_df.program_subdomain == 'fe')) | (lesson_df.visits == ds_max)

In [45]:
most_traffic = lesson_df[mask] # applying mask
most_traffic

,program_subdomain,path,stem,tail,content,visits
0,ds,classification/overview,classification,overview,None,1785
520,fe,content/html-css,content,html-css,html-css,2
523,java,javascript-i/introduction/working-with-data-ty...,javascript-i,working-with-data-types-operators-and-variables,None,7094
1862,php,content/html-css,content,html-css,html-css,262


#### Data Science
> - classification/overview

#### Front End
> - content/html-css

#### Java
> - javascript-i/introduction/working-with-data-types-operators-and-variables


#### PHP Full Stack:
> - content/html-css

### 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
> Looking at the Data Science cohorts:

In [46]:
# Creating dataframe based on grouping by cohort_name for path hits
cohort_df = pd.DataFrame(ds_df.groupby('path').cohort_name.value_counts())

In [47]:
# Renaming to be accurate to info and resetting index
cohort_df = cohort_df.rename(columns = {'cohort_name': 'visits'}, level = 0)

In [48]:
cohort_df

visits
path                                               cohort_name        
1-fundamentals/1.1-intro-to-data-science           Bayes           640
                                                   Curie           461
                                                   Darden          460
                                                   Florence         64
                                                   Easley            8
...                                                                ...
timeseries/working-with-time-series-data           Bayes             7
                                                   Curie             3
timeseries/working-with-time-series-data-with-p... Darden           11
                                                   Bayes             4
                                                   Curie             2

[1470 rows x 1 columns]

In [49]:
# Loop to get minimum value for each lesson and the corresponding cohort
path_dict = {'lesson': [],'cohort':[], 'visits': []}
for lesson in cohort_df.index.get_level_values(0).unique(): # due to multi-index need get_level_values
    min_vis = cohort_df[cohort_df.index.get_level_values(0) == lesson].visits.values # min visits per lesson
    min_coh = cohort_df[(cohort_df.visits == min_vis.min())\ # getting corresponding cohort with min visits on a lesson
                        & (cohort_df.index.get_level_values(0) == lesson)].index.get_level_values(1).values[0]
    
    path_dict['lesson'].append(lesson) # appending lesson (path)
    path_dict['cohort'].append(min_coh) # cohort with min visits to lesson
    path_dict['visits'].append(min_vis.min()) # number of visits to lesson

In [50]:
# Creating df of each lesson, the cohort with the least visits to that lesson
# and the number of times that cohort visited the lesson
path_df = pd.DataFrame.from_dict(path_dict)

In [53]:
path_df.head()

,lesson,cohort,visits
0,1-fundamentals/1.1-intro-to-data-science,Easley,8
1,1-fundamentals/1.2-data-science-pipeline,Curie,146
2,1-fundamentals/1.3-pipeline-demo,Curie,60
3,1-fundamentals/2.1-excel-overview,Easley,16
4,1-fundamentals/2.1-spreadsheets-overview,Easley,2


In [274]:
# Which cohort overall accessed any given lesson the least
path_df.cohort.value_counts(normalize = True)

Bayes       0.380769
Curie       0.230769
Darden      0.230769
Florence    0.088462
Easley      0.069231
Name: cohort, dtype: float64

### Across the Data Science Program:
> - There is almost a 40% chance that on a given lesson, the Bayes cohort will be the least active visiting it.

### 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [60]:
# Creating df grouped by user_id and utilizing the count_helper column to get
# the number of occurrances of that user_id
id_df = pd.DataFrame(df.groupby('user_id').count_helper.sum().sort_values())
id_df

,count_helper
user_id,
212,1
348,1
169,2
85,2
177,2
...,...
581,4821
314,5796
53,8030


In [64]:
# Checking lowest 10%
bottom_ten = id_df.count_helper.quantile(.10)

In [66]:
# Recasting df as only the users in the bottom 10% of activity
id_df = id_df[id_df.count_helper < bottom_ten].reset_index()

In [275]:
rare_users = id_df.user_id # series users in the bottom 10% of activity
rare_users

0     212
1     348
2     169
3      85
4     177
     ... 
83    813
84    972
85    740
86    322
87    971
Name: user_id, Length: 88, dtype: int64

In [80]:
# Calling base df and subsetting it so it only contains our rare_users
rare_df = df[df.user_id.isin(rare_users)]
rare_df.cohort_name.value_counts() # Checking which cohorts have the most inactive users

Oberon       233
Staff        128
Quincy        67
Arches        61
Sequoia       55
Niagara       52
Joshua        49
Deimos        44
Yosemite      44
Pinnacles     43
Ike           40
Hampton       36
Ulysses       33
Lassen        31
Bayes         27
Europa        25
Olympic       23
Curie         22
Teddy         19
Jupiter       19
Badlands      18
Kings         16
Marco         13
Apex          12
Glacier       12
Darden        11
Zion          10
Hyperion      10
Andromeda      6
Mammoth        5
Neptune        4
Apollo         4
Denali         2
Name: cohort_name, dtype: int64

In [83]:
# Which program has the bottom 10% of users?
rare_df.program_subdomain.value_counts()

java    755
php     355
ds       60
fe        4
Name: program_subdomain, dtype: int64

In [84]:
# Does the time frame give us any insights?
rare_df.datetime.min(), rare_df.datetime.max()

(Timestamp('2018-01-26 00:00:00'), Timestamp('2021-04-21 00:00:00'))

In [87]:
# When did most of these users begin their program
rare_df.start_date.value_counts()

2021-04-12    233
2014-02-04    189
2017-06-05     67
2017-09-27     55
2016-10-26     52
2016-03-08     49
2019-09-16     44
2018-11-05     44
2017-03-27     43
2016-01-20     40
2015-09-22     36
2018-03-05     33
2016-07-18     31
2019-08-19     27
2019-11-04     25
2017-02-06     23
2020-02-03     22
2018-01-08     19
2020-09-21     19
2014-06-04     18
2016-05-23     16
2021-01-02     13
2015-06-05     12
2020-02-24     12
2020-07-13     11
2019-01-22     10
2020-05-26     10
2019-03-18      6
2016-09-26      5
2021-03-13      4
2015-03-30      4
2014-10-20      2
Name: start_date, dtype: int64

Users who started in 2021 are catagroized as bottom 10% due to the recency of their program in comparison to the amount of time other users have had access to the lessons.

In [89]:
# Filtering out those who just started
rare_df = rare_df[rare_df.start_date != '2021-04-12']

In [90]:
# Once removed recent starts, when did most of these users begin their program
rare_df.start_date.value_counts()

2014-02-04    189
2017-06-05     67
2017-09-27     55
2016-10-26     52
2016-03-08     49
2018-11-05     44
2019-09-16     44
2017-03-27     43
2016-01-20     40
2015-09-22     36
2018-03-05     33
2016-07-18     31
2019-08-19     27
2019-11-04     25
2017-02-06     23
2020-02-03     22
2020-09-21     19
2018-01-08     19
2014-06-04     18
2016-05-23     16
2021-01-02     13
2015-06-05     12
2020-02-24     12
2020-07-13     11
2019-01-22     10
2020-05-26     10
2019-03-18      6
2016-09-26      5
2015-03-30      4
2021-03-13      4
2014-10-20      2
Name: start_date, dtype: int64

#### Hardly Active Students:
> - Oberron cohort has the 266 students in the bottom 10% of activity
> - Most inactive students belong in the Java program
> - Almost half of the users in the bottom 10% of activity started in 2021
>    - Also large ammount of users who are inactive have already finished their programs.

### 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

### 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [94]:
# Making sure end_date is a datetime object
df.end_date = pd.to_datetime(df.end_date)

In [95]:
# Checking timeframe on datetime of entries
df.datetime.min(), df.datetime.max()

(Timestamp('2018-01-26 00:00:00'), Timestamp('2021-04-21 00:00:00'))

In [110]:
# Checking time frame of end_dates for the programs
df.end_date.min(), df.end_date.max()

(Timestamp('2014-02-04 00:00:00'), Timestamp('2021-09-28 00:00:00'))

In [98]:
# Creating df of users who are accessing lessons after their graduation
diff_time = df[df.datetime > df.end_date][['path', 'user_id', 'end_date', 'datetime']]

In [107]:
# Again, making sure end_date stays as datetime object
diff_time['end_date'] = pd.to_datetime(diff_time['end_date'])

In [108]:
diff_time

,path,user_id,end_date,datetime
1,java-ii/object-oriented-programming,1,2016-02-06,2018-01-26
2,slides/object_oriented_programming,1,2016-02-06,2018-01-26
3,javascript-i/functions,1,2016-02-06,2018-01-26
6,teams/13,1,2016-02-06,2018-01-27
7,javascript-i/conditionals,1,2016-02-06,2018-01-29
...,...,...,...,...
716960,javascript-i/functions,762,2021-03-30,2021-04-20
716963,javascript-ii/es6,762,2021-03-30,2021-04-20
716964,javascript-ii/promises,762,2021-03-30,2021-04-20
716966,spring/setup,773,2021-03-30,2021-04-20


In [142]:
# Making list of the difference between the time a user accesses the lesson and the time they graduated
diffs = (diff_time['datetime'] - diff_time['end_date']).dt.days

In [145]:
# Using list to create column for difference in days
diff_time['day_diff'] = diffs

In [149]:
diff_time

,path,user_id,end_date,datetime,day_diff
1,java-ii/object-oriented-programming,1,2016-02-06,2018-01-26,720
2,slides/object_oriented_programming,1,2016-02-06,2018-01-26,720
3,javascript-i/functions,1,2016-02-06,2018-01-26,720
6,teams/13,1,2016-02-06,2018-01-27,721
7,javascript-i/conditionals,1,2016-02-06,2018-01-29,723
...,...,...,...,...,...
716960,javascript-i/functions,762,2021-03-30,2021-04-20,21
716963,javascript-ii/es6,762,2021-03-30,2021-04-20,21
716964,javascript-ii/promises,762,2021-03-30,2021-04-20,21
716966,spring/setup,773,2021-03-30,2021-04-20,21


Lets say it takes 3 months on average after codeup to get hired

In [182]:
# Creating a month_diff column using the day_diff
diff_time['month_diff'] = diff_time.day_diff / 30

# Setting difftime to entries 3 or more months post-graduation
diff_time = diff_time[diff_time.month_diff >= 3]

In [183]:
diff_time

,path,user_id,end_date,datetime,day_diff,month_diff
1,java-ii/object-oriented-programming,1,2016-02-06,2018-01-26,720,24.000000
2,slides/object_oriented_programming,1,2016-02-06,2018-01-26,720,24.000000
3,javascript-i/functions,1,2016-02-06,2018-01-26,720,24.000000
6,teams/13,1,2016-02-06,2018-01-27,721,24.033333
7,javascript-i/conditionals,1,2016-02-06,2018-01-29,723,24.100000
...,...,...,...,...,...,...
664609,python/control-structures,692,2021-01-12,2021-04-21,99,3.300000
664610,python/data-types-and-variables,692,2021-01-12,2021-04-21,99,3.300000
664611,classification/overview,692,2021-01-12,2021-04-21,99,3.300000
664614,classification/project,692,2021-01-12,2021-04-21,99,3.300000


In [184]:
# Creating df grouping by path to check how many visits a given lesson has
grad_visits = pd.DataFrame(diff_time.groupby('path').path.value_counts())
grad_visits.rename(columns = {'path' : 'visits'}, inplace = True) # renaming the value column

,,visits
path,path,
1-fundamentals/1-fundamentals-overview,1-fundamentals/1-fundamentals-overview,56
1-fundamentals/1-intro-to-data-science,1-fundamentals/1-intro-to-data-science,4
1-fundamentals/1.1-intro-to-data-science,1-fundamentals/1.1-intro-to-data-science,555
1-fundamentals/1.2-data-science-pipeline,1-fundamentals/1.2-data-science-pipeline,89
1-fundamentals/1.3-pipeline-demo,1-fundamentals/1.3-pipeline-demo,62
...,...,...
web-design/ui/color,web-design/ui/color,78
web-design/ui/typography,web-design/ui/typography,92
web-design/ui/visuals,web-design/ui/visuals,56


In [185]:
# Dropping one level of the multi-index as it is repeated
grad_visits.index = grad_visits.index.droplevel(level = 0)

In [186]:
grad_visits

,visits
path,
1-fundamentals/1-fundamentals-overview,56
1-fundamentals/1-intro-to-data-science,4
1-fundamentals/1.1-intro-to-data-science,555
1-fundamentals/1.2-data-science-pipeline,89
1-fundamentals/1.3-pipeline-demo,62
...,...
web-design/ui/color,78
web-design/ui/typography,92
web-design/ui/visuals,56


In [187]:
# Getting rid of path-index
grad_visits = grad_visits.reset_index()

In [193]:
# Finding the 3 most visited post-grad lessons
grad_visits.sort_values('visits', ascending = False).head(3)

,path,visits
1206,spring/fundamentals/repositories,1311
1042,jquery/ajax/weather-map,1305
1210,spring/fundamentals/views,1252


#### Most Visited Lessons Post-Graduation:
> - spring/fundamentals/repositories
> - jquery/ajax/weather-map
> - spring/fundamentals/views

### 7. Which lessons are least accessed?

In [207]:
# Creating grouped df by program_subdomain and path value_counts
lesson_df = pd.DataFrame(df.groupby('program_subdomain').path.value_counts())

In [208]:
# Renaming to be accurate to info and reseting multi-index
lesson_df = lesson_df.rename(columns = {'path': 'visits'}, level = 0).reset_index()

In [209]:
# Getting min values for each subdomain for each max value
ds_min = lesson_df[(lesson_df.program_subdomain == 'ds')].visits.min()
php_min = lesson_df[(lesson_df.program_subdomain == 'php')].visits.min()
fe_min = lesson_df[(lesson_df.program_subdomain == 'fe')].visits.min()
java_min = lesson_df[(lesson_df.program_subdomain == 'java')].visits.min()

In [210]:
# Masking to find the min across each program
mask = (lesson_df.visits == php_min)  | (lesson_df.visits == java_min) | ((lesson_df.visits == fe_min) & (lesson_df.program_subdomain == 'fe')) | (lesson_df.visits == ds_min)

In [211]:
# Creating subpath column for lesson_df
lesson_df['subpath'] = lesson_df.path.apply(lambda x: x.split('/'))

# Recasting it with chosen columns
lesson_df = lesson_df[['program_subdomain', 'path', 'subpath', 'visits']]
lesson_df.head()

,program_subdomain,path,subpath,visits
0,ds,classification/overview,"[classification, overview]",1785
1,ds,1-fundamentals/1.1-intro-to-data-science,"[1-fundamentals, 1.1-intro-to-data-science]",1633
2,ds,sql/mysql-overview,"[sql, mysql-overview]",1424
3,ds,fundamentals/intro-to-data-science,"[fundamentals, intro-to-data-science]",1413
4,ds,6-regression/1-overview,"[6-regression, 1-overview]",1124


In [212]:
# Getting stems and destination from paths
sub_lens = [] # lengths of each path
large_paths = [] # paths larger than 2 /'s deep
starter = [] # path stem
final = [] # path tail
lesson = [] # tails where the stem != 'content'
for path in lesson_df.subpath.values:
    sub_lens.append(len(path))
    starter.append(path[0])
    final.append(path[-1])
    if len(path) > 2:
        large_paths.append(path)
    if path[0] == 'content': # checking for content stem
        lesson.append(path[-1])
    else:
        lesson.append('None') # if stem != content cast as None

In [224]:
# Creating columns for stems, tails and content containing entries
lesson_df['stem'] = starter
lesson_df['final'] = final
lesson_df['content'] = lesson
lesson_df = lesson_df[['program_subdomain', 'path', 'stem', 'final', 'content', 'visits']]

In [225]:
# Masking for least visited paths
least_traffic = lesson_df[mask]
least_traffic

,program_subdomain,path,stem,final,content,visits
469,ds,12-distributed-ml/3-getting-started,12-distributed-ml,3-getting-started,None,1
470,ds,12-distributed-ml/4-acquire,12-distributed-ml,4-acquire,None,1
471,ds,12-distributed-ml/5-inspect,12-distributed-ml,5-inspect,None,1
472,ds,12-distributed-ml/6.1-prepare-part-1,12-distributed-ml,6.1-prepare-part-1,None,1
473,ds,12-distributed-ml/6.2-prepare-part-2,12-distributed-ml,6.2-prepare-part-2,None,1
...,...,...,...,...,...,...
2473,php,spring/security,spring,security,None,1
2474,php,student/create,student,create,None,1
2475,php,students/468/notes,students,notes,None,1
2476,php,students/units/75/sub_units/268,students,268,None,1


In [227]:
# Creating dfs for each program for the lesat number of visits to lessons
ds_least = least_traffic[least_traffic.program_subdomain == 'ds']
php_least = least_traffic[least_traffic.program_subdomain == 'php']
fe_least = least_traffic[least_traffic.program_subdomain == 'fe']
java_least = least_traffic[least_traffic.program_subdomain == 'java']

In [240]:
# top 3 least visited Data Science
ds_least.path.value_counts()[:3]

12-distributed-ml/3-getting-started        1
java-i/console-io                          1
appendix/univariate_regression_in_excel    1
Name: path, dtype: int64

In [236]:
# top 3 least visited PHP
php_least.path.value_counts()[:3]

1-fundamentals/2.4-more-excel-features     1
content/php_iii/alternative-syntax.html    1
content/mysql/clauses/order-by.html        1
Name: path, dtype: int64

In [237]:
# top 3 least visited Front End Dev
fe_least.path.value_counts()[:3]

content/html-css/gitbook/images/favicon.ico    1
content/html-css/introduction.html             1
Name: path, dtype: int64

In [238]:
# top 3 least visited Java Full Stack
java_least.path.value_counts()[:3]

10-anomaly-detection/2-continuous-probabilistic-methods.ipynb    1
html-css/introduction:page-source                                1
further-reading/javascript/array-splice                          1
Name: path, dtype: int64

#### Least Visited Lessons Across Programs:
##### Data Science
> - 12-distributed-ml/3-getting-started
> - java-i/console-io
> - appendix/univariate_regression_in_excel

##### Front End
> - content/html-css/gitbook/images/favicon.ico
> - content/html-css/introduction.html

##### Java
> - 10-anomaly-detection/2-continuous-probabilistic-methods.ipynb
> - html-css/introduction:page-source
> - further-reading/javascript/array-splice

##### PHP Full Stack:
> - 1-fundamentals/2.4-more-excel-features
> - content/php_iii/alternative-syntax.html
> - content/mysql/clauses/order-by.html